<a href="https://colab.research.google.com/github/sheldonkemper/bank_of_england/blob/tidy_up_preprocessing_notebook/ct_preprocessing_ubs_pairing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [169]:
"""
===================================================
Author: Chiaki Tachikawa
Role: Data Science Lead, Bank of England Employer Project (Quant Collective)
LinkedIn: https://www.linkedin.com/in/chiaki-tachikawa
Date: 2025-02-13
Version: 1.1

Description:
    This notebook implements a system for cleaning and exporting transcript data for the Bank of England project. The workflow includes:
    - Importing necessary libraries and downloading NLTK data.
    - Defining and applying a `preprocessor` function to clean and tokenize text data.
    - Reading and preprocessing various CSV files containing transcript data.
    - Exporting the preprocessed data to new CSV files for further analysis.

===================================================
"""

'\n===================================================\nAuthor: Chiaki Tachikawa\nRole: Data Science Lead, Bank of England Employer Project (Quant Collective)\nLinkedIn: https://www.linkedin.com/in/chiaki-tachikawa\nDate: 2025-02-13\nVersion: 1.1\n\nDescription:\n    This notebook implements a system for cleaning and exporting transcript data for the Bank of England project. The workflow includes:\n    - Importing necessary libraries and downloading NLTK data.\n    - Defining and applying a `preprocessor` function to clean and tokenize text data.\n    - Reading and preprocessing various CSV files containing transcript data.\n    - Exporting the preprocessed data to new CSV files for further analysis.\n\n===================================================\n'

# **Library**

In [170]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
nltk.download("stopwords")
nltk.download("punkt")
nltk.download("punkt_tab")
nltk.download('wordnet')
from nltk.tokenize import word_tokenize, sent_tokenize
from collections import Counter
import regex as re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from google.colab import drive


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


# **Function**

preprocessor function : The function modifies the DataFrame data in place, adding two new columns (col1 and col2) with preprocessed text.


Input:
  - name of dataframe
  - name of column which contains the text to clean
  - name of column which is tokenized
  - name of column which is cleaned

In [171]:
#create function to preprocess data
def preprocessor (data, col, col1,col2):
  #Copy col1umn
  data[col1]=data[col]
  data[col2]=data[col]


  #Adding column1
  #Lower the lettercase
  data[col1] = data[col1].str.lower()

  #Remove stop words
  stop_words = set(stopwords.words("english"))
  data[col1] = data[col1].apply(lambda x: " ".join([word for word in str(x).split() if word not in (stop_words)]))

  #Tokenize the word
  data[col1] = data[col1].apply(nltk.word_tokenize)

  #Remove numbers
  data[col1] = data[col1].apply(lambda x: [word for word in x if not word.isdigit()])

  #remove symbol from comments
  data[col1] = data[col1].apply(lambda x: [word for word in x if x!=""])

  #remove short word
  data[col1] = data[col1].apply(lambda x: [word for word in x if len(word)>2])

  #remove symbols
  data[col1] = data[col1].apply (lambda x: [re.sub(r"[^a-z]", "", word) for word in x])

  #lemmatization
  lemmatizer = WordNetLemmatizer()
  data[col1] = data[col1].apply(lambda x: [lemmatizer.lemmatize(word) for word in x])



  #Adding column2
  #Lower the lettercase
  data[col2] = data[col2].str.lower()

  #Remove stop words
  stop_words = set(stopwords.words("english"))
  data[col2] = data[col2].apply(lambda x: " ".join([word for word in str(x).split() if word not in (stop_words)]))

  #remove symbols
  data[col2] = data[col2].apply (lambda x: [re.sub(r"[.,'?]", "", x)])

  return


find_row: This function searches upwards from the given current_row_num in the DataFrame df to find the first row where the value in column "M" is "A". It returns the index of that row. If no such row is found, it returns 0

In [172]:
def find_row (df, col, current_row_num):
  #list_name=[]
  i = current_row_num-1
  while i > 0:
    if df[col][i] == "A":
      break
    else:
      i-=1
  return i

find_row_empty: This function searches upwards from the given current_row_num to find the first row where col1 has the value "A" and col2 is not an empty list. It returns the index of that row.

In [173]:
def find_row_empty (df, col1, col2, current_row_num):
  #list_name=[]
  i = current_row_num-1
  while i > 0:
    if df[col1][i] == "A" and df[col2][i] != []:
      break
    else:
      i-=1
  return i

In [174]:
def create_ques_num_column (data, new_col,marker_col):
  #Create question number column
  data[new_col]=None
  #set global var to count question number
  num = 0
  #if Q was found, num adds 1 otherwise none
  for i in data.index:
    if data.loc[i,marker_col] is not np.NaN:
      data.at[i,new_col]=num
      num +=1
    else:
      continue

In [175]:
# Function to extract names
def extract_name(full_string):
    return full_string.split(',')[0]

In [176]:
#check if there is "A" before "Q" from the current location
def find_last_a (df, col, current_row_num):
  #list_name=[]
  i = current_row_num-1
  while i > 0:
    if df[col][i] =="UBS":
      j = i-1
      while j > 0:
        if df[col][j]!="UBS":
          pass
        else:
          break
        j-=1
      break
    else:
      i-=1
  return i

## **Data**

In [177]:
#drive.mount('/content/drive')

In [178]:
#!ls"/content/bank_of_england/data/preprocessed_data/Archived/jpmorgan_qa_section_preprocessed.csv"

In [179]:
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


JP Morgan QA section

In [180]:
#Defining qa_data
qa_data = pd.read_csv("/content/ubs_data.csv")
qa_data.head()

,speaker,job_title,utterance,call_date,financial_quarter,source_file,ex_dummy,category
0,Chis Hallam,Goldman,"Chis Hallam, Goldman Sachs Yes. Good morning, ...",25 April 2023,1Q23,1q23-earnings-call-remarks.pdf,NaN,Goldman
1,Sergio P. Ermotti,UBS,"Okay. Thank you. On capital requirements, you ...",25 April 2023,1Q23,1q23-earnings-call-remarks.pdf,NaN,Goldman
2,Chris Hallam,Goldman Sachs,Very clear. Thanks.,25 April 2023,1Q23,1q23-earnings-call-remarks.pdf,NaN,Goldman
3,Kian Abouhossein,JPMorgan,Yeah. Thanks. Just two questions. The first on...,25 April 2023,1Q23,1q23-earnings-call-remarks.pdf,NaN,JPMorgan
4,Sergio P. Ermotti,UBS,"So, Sarah, take the first question. I'll take ...",25 April 2023,1Q23,1q23-earnings-call-remarks.pdf,NaN,JPMorgan


In [181]:
qa_data["question"] = None
qa_data["answer"] = None
current_bank=qa_data["category"][0]
question=""
answer=""
for i in range(len(qa_data)):
  if qa_data["job_title"][i]!="UBS" and current_bank==qa_data["category"][i]:
    question += str(qa_data["utterance"][i])
  elif qa_data["job_title"][i]=="UBS" and current_bank==qa_data["category"][i]:
    answer += str(qa_data["utterance"][i])
  elif qa_data["job_title"][i]!="UBS" and current_bank!=qa_data["category"][i]:
    current_bank=qa_data["job_title"][i]
    qa_data.at[i-1,"question"]=question
    qa_data.at[i-1,"answer"]=answer
    question = str(qa_data["utterance"][i])
    answer = ""
  else:
    continue

In [182]:
"""
qa_data["question"]=None
qa_data["answer"] =None

indices_to_drop = []

for i in range(len(qa_data)):
  if qa_data["job_title"][i]!="UBS":
     qa_data.at[i,"question"] = qa_data.loc[i,"utterance"]
  else:
    if i >0 and tuple(qa_data.iloc[i-1])in[tuple(qa_data.iloc[i])]:  #qa_data.isin(qa_data[i-1]):
      qa_data.at[i-1,"answer"] = qa_data.loc[i,"utterance"]
    else:
      if i>1:
        qa_data.at[i-2,"answer"] = qa_data.loc[i,"utterance"]
    indices_to_drop.append(i)

qa_data.drop(indices_to_drop, inplace=True)
qa_data.reset_index(drop=True, inplace=True)
"""

'\nqa_data["question"]=None\nqa_data["answer"] =None\n\nindices_to_drop = []\n\nfor i in range(len(qa_data)):\n  if qa_data["job_title"][i]!="UBS":\n     qa_data.at[i,"question"] = qa_data.loc[i,"utterance"]\n  else:\n    if i >0 and tuple(qa_data.iloc[i-1])in[tuple(qa_data.iloc[i])]:  #qa_data.isin(qa_data[i-1]):\n      qa_data.at[i-1,"answer"] = qa_data.loc[i,"utterance"]\n    else:\n      if i>1:\n        qa_data.at[i-2,"answer"] = qa_data.loc[i,"utterance"]\n    indices_to_drop.append(i) \n\nqa_data.drop(indices_to_drop, inplace=True)\nqa_data.reset_index(drop=True, inplace=True)\n'

In [191]:
qa_data.head(50)

,index,speaker,job_title,utterance,call_date,financial_quarter,source_file,ex_dummy,category,question,answer
0,0,Chis Hallam,Goldman,"Chis Hallam, Goldman Sachs Yes. Good morning, ...",25 April 2023,1Q23,1q23-earnings-call-remarks.pdf,NaN,Goldman,None,None
1,1,Sergio P. Ermotti,UBS,"Okay. Thank you. On capital requirements, you ...",25 April 2023,1Q23,1q23-earnings-call-remarks.pdf,NaN,Goldman,None,None
2,2,Chris Hallam,Goldman Sachs,Very clear. Thanks.,25 April 2023,1Q23,1q23-earnings-call-remarks.pdf,NaN,Goldman,"Chis Hallam, Goldman Sachs Yes. Good morning, ...","Okay. Thank you. On capital requirements, you ..."
3,3,Kian Abouhossein,JPMorgan,Yeah. Thanks. Just two questions. The first on...,25 April 2023,1Q23,1q23-earnings-call-remarks.pdf,NaN,JPMorgan,None,None
4,4,Sergio P. Ermotti,UBS,"So, Sarah, take the first question. I'll take ...",25 April 2023,1Q23,1q23-earnings-call-remarks.pdf,NaN,JPMorgan,None,None
5,5,Sarah Youngwood,UBS,"So, when we give you the 74%, we focused inten...",25 April 2023,1Q23,1q23-earnings-call-remarks.pdf,NaN,JPMorgan,None,None
6,6,Kian Abouhossein,JPMorgan,Sorry if I may. The 74% also includes - but do...,25 April 2023,1Q23,1q23-earnings-call-remarks.pdf,NaN,JPMorgan,None,None
7,7,Sarah Youngwood,UBS,That's right. The initial PPA comes into the C...,25 April 2023,1Q23,1q23-earnings-call-remarks.pdf,NaN,JPMorgan,None,None
8,8,Kian Abouhossein,JPMorgan,Ok.,25 April 2023,1Q23,1q23-earnings-call-remarks.pdf,NaN,JPMorgan,None,None
9,9,Sergio P. Ermotti,UBS,"So on client retention, I – maybe let me reite...",25 April 2023,1Q23,1q23-earnings-call-remarks.pdf,NaN,JPMorgan,Yeah. Thanks. Just two questions. The first on...,"So, Sarah, take the first question. I'll take ..."


In [192]:
filtered_data= qa_data.dropna(subset=["question"])

In [193]:
filtered_data.reset_index(drop=True, inplace=True)

In [199]:
filtered_data=filtered_data.drop(columns="ex_dummy")

In [200]:
filtered_data.head()

,speaker,job_title,utterance,call_date,financial_quarter,source_file,category,question,answer
0,Chris Hallam,Goldman Sachs,Very clear. Thanks.,25 April 2023,1Q23,1q23-earnings-call-remarks.pdf,Goldman,"Chis Hallam, Goldman Sachs Yes. Good morning, ...","Okay. Thank you. On capital requirements, you ..."
1,Sergio P. Ermotti,UBS,"So on client retention, I – maybe let me reite...",25 April 2023,1Q23,1q23-earnings-call-remarks.pdf,JPMorgan,Yeah. Thanks. Just two questions. The first on...,"So, Sarah, take the first question. I'll take ..."
2,Alastair Ryan,Bank of America,Thank you.,25 April 2023,1Q23,1q23-earnings-call-remarks.pdf,Bank of America,"Yeah. Thank you. Good morning. Welcome back, S...","Thank you, Ryan. It is good to be back to inte..."
3,Flora Bocahut,Jefferies,Thank you.,25 April 2023,1Q23,1q23-earnings-call-remarks.pdf,Jefferies,Yes. Good morning. The first question I wanted...,"So, on the first question in terms of the tren..."
4,Andrew Coombs,Citi,Thank you.,25 April 2023,1Q23,1q23-earnings-call-remarks.pdf,Citi,"Good morning. Two questions. Firstly, just on ...","So, on the first quarter or the first question..."


In [202]:
filtered_data.to_csv("/content/ubs_qa_df_ver1.csv", index=False)

In [42]:
"""
#ascend df by financial_quarter
quarter_order = ["1Q23", "2Q23","3Q23","4Q23","1Q24","2Q24","3Q24","4Q24"]
qa_data["financial_quarter"] = pd.Categorical(qa_data["financial_quarter"], categories=quarter_order, ordered=True)
qa_data = qa_data.sort_values("financial_quarter", kind="mergesort")
"""

'\n#ascend df by financial_quarter\nquarter_order = ["1Q23", "2Q23","3Q23","4Q23","1Q24","2Q24","3Q24","4Q24"]\nqa_data["financial_quarter"] = pd.Categorical(qa_data["financial_quarter"], categories=quarter_order, ordered=True)\nqa_data = qa_data.sort_values("financial_quarter", kind="mergesort")\n'

In [43]:
#preprocessing data
preprocessor(qa_data, "utterance", "question_tokenised_data", "question_cleaned")
#preprocessor(qa_data,"utterance","answer_tokenised_data","answer_cleaned")

In [44]:
#remove operater
#qa_data = qa_data.loc[qa_data["speaker"]!="Operator"]

#remove less than 20 words
qa_data["count"] = qa_data["question_tokenised_data"].apply(lambda x: len(x))
qa_data = qa_data.loc[qa_data["count"]>20]
qa_data.head()


,speaker,job_title,utterance,call_date,financial_quarter,source_file,ex_dummy,category,question_tokenised_data,question_cleaned,count
0,Chis Hallam,Goldman,"Chis Hallam, Goldman Sachs Yes. Good morning, ...",25 April 2023,1Q23,1q23-earnings-call-remarks.pdf,NaN,Goldman,"[chi, hallam, goldman, sachs, yes, good, morni...",[chis hallam goldman sachs yes good morning ev...,96
1,Sergio P. Ermotti,UBS,"Okay. Thank you. On capital requirements, you ...",25 April 2023,1Q23,1q23-earnings-call-remarks.pdf,NaN,Goldman,"[okay, thank, you, capital, requirement, know,...",[okay thank you capital requirements know situ...,73
3,Kian Abouhossein,JPMorgan,Yeah. Thanks. Just two questions. The first on...,25 April 2023,1Q23,1q23-earnings-call-remarks.pdf,NaN,JPMorgan,"[yeah, thanks, two, question, first, one, rela...",[yeah thanks two questions first one related b...,65
5,Sarah Youngwood,UBS,"So, when we give you the 74%, we focused inten...",25 April 2023,1Q23,1q23-earnings-call-remarks.pdf,NaN,JPMorgan,"[give, focused, intentionally, viewed, economi...",[so give 74% focused intentionally viewed econ...,27
9,Sergio P. Ermotti,UBS,"So on client retention, I – maybe let me reite...",25 April 2023,1Q23,1q23-earnings-call-remarks.pdf,NaN,JPMorgan,"[client, retention, maybe, let, reiterate, loo...",[client retention – maybe let reiterate look i...,80


In [29]:
#reset index
qa_data.reset_index(drop=True, inplace=True)
qa_data.head(20)

,speaker,job_title,utterance,call_date,financial_quarter,source_file,ex_dummy,category,question_tokenised_data,question_cleaned,count,analyst,analyst_bank,metadata_question,question_num
0,Chis Hallam,Goldman,"Chis Hallam, Goldman Sachs Yes. Good morning, ...",25 April 2023,1Q23,1q23-earnings-call-remarks.pdf,NaN,Goldman,"[chi, hallam, goldman, sachs, yes, good, morni...",[chis hallam goldman sachs yes good morning ev...,96,[],[],[],[]
1,Sergio P. Ermotti,UBS,"Okay. Thank you. On capital requirements, you ...",25 April 2023,1Q23,1q23-earnings-call-remarks.pdf,NaN,Goldman,"[okay, thank, you, capital, requirement, know,...",[okay thank you capital requirements know situ...,73,[],[],[],[]
2,Kian Abouhossein,JPMorgan,Yeah. Thanks. Just two questions. The first on...,25 April 2023,1Q23,1q23-earnings-call-remarks.pdf,NaN,JPMorgan,"[yeah, thanks, two, question, first, one, rela...",[yeah thanks two questions first one related b...,65,[],[],[],[]
3,Sarah Youngwood,UBS,"So, when we give you the 74%, we focused inten...",25 April 2023,1Q23,1q23-earnings-call-remarks.pdf,NaN,JPMorgan,"[give, focused, intentionally, viewed, economi...",[so give 74% focused intentionally viewed econ...,27,"[Sergio P. Ermotti, Kian Abouhossein]","[UBS, JPMorgan]","[Okay. Thank you. On capital requirements, you...",[]
4,Sergio P. Ermotti,UBS,"So on client retention, I – maybe let me reite...",25 April 2023,1Q23,1q23-earnings-call-remarks.pdf,NaN,JPMorgan,"[client, retention, maybe, let, reiterate, loo...",[client retention – maybe let reiterate look i...,80,[Sarah Youngwood],[UBS],"[So, when we give you the 74%, we focused inte...",[]
5,Alastair Ryan,Bank of America,"Yeah. Thank you. Good morning. Welcome back, S...",25 April 2023,1Q23,1q23-earnings-call-remarks.pdf,NaN,Bank of America,"[yeah, thank, you, good, morning, welcome, bac...",[yeah thank you good morning welcome back serg...,74,[],[],[],[]
6,Sergio P. Ermotti,UBS,"Thank you, Ryan. It is good to be back to inte...",25 April 2023,1Q23,1q23-earnings-call-remarks.pdf,NaN,Bank of America,"[thank, you, ryan, good, back, interact, well,...",[thank you ryan good back interact well look y...,106,"[Sergio P. Ermotti, Kian Abouhossein, Sarah Yo...","[UBS, JPMorgan, UBS, UBS, Bank of America]","[Okay. Thank you. On capital requirements, you...",[]
7,Flora Bocahut,Jefferies,Yes. Good morning. The first question I wanted...,25 April 2023,1Q23,1q23-earnings-call-remarks.pdf,NaN,Jefferies,"[yes, good, morning, first, question, wanted, ...",[yes good morning first question wanted ask re...,96,[],[],[],[]
8,Sergio P. Ermotti,UBS,"Yeah. On the client side. I mean, if you look ...",25 April 2023,1Q23,1q23-earnings-call-remarks.pdf,NaN,Jefferies,"[yeah, client, side, mean, look, example, asia...",[yeah client side mean look at example asia th...,146,"[Sergio P. Ermotti, Kian Abouhossein, Sarah Yo...","[UBS, JPMorgan, UBS, UBS, Bank of America, UBS...","[Okay. Thank you. On capital requirements, you...",[]
9,Andrew Coombs,Citi,"Good morning. Two questions. Firstly, just on ...",25 April 2023,1Q23,1q23-earnings-call-remarks.pdf,NaN,Citi,"[good, morning, two, question, firstly, slide,...",[good morning two questions firstly slide 10 r...,111,[],[],[],[]


In [45]:
#adding new columns
qa_data["analyst"]=None
qa_data["analyst_bank"]=None
qa_data["metadata_question"]=None
qa_data["question_num"]=None

#

for i in qa_data.index:
    name_list=[]
    bank_list=[]
    question_list=[]
    quest_num_list=[]
    #If the value in the "job_title" column at row i is "bank name", then set the values of "analyst", "analyst_bank", "metadata_question", and "question_num" columns at row i to "x".
    if qa_data.loc[i,"job_title"]!="UBS":
      qa_data.at[i,"analyst"]="x"
      qa_data.at[i,"analyst_bank"]="x"
      qa_data.at[i,"metadata_question"]="x"
      qa_data.at[i,"question_num"]="x"
    elif find_last_a(qa_data,"job_title",i) >=0 and qa_data.loc[i-1,"job_title"] =="UBS":
      name_list.append(qa_data["speaker"][i-1])
      bank_list.append(qa_data["job_title"][i-1])
      question_list.append(qa_data["utterance"][i-1])
    #  quest_num_list.append(str(qa_data["question_number_inline"][i-1]))
    elif find_last_a(qa_data,"job_title",i) ==0 and qa_data.loc[i-1,"job_title"] =="UBS":
      qa_data.at[i,"analyst"] = qa_data.at[i-1,"analyst"]
      qa_data.at[i, "analyst_bank"] = qa_data.at[i-1,"analyst_bank"]
      qa_data.at[i,"metadata_question"]=qa_data.at[i-1, "metadata_question"]
      qa_data.at[i,"question_num"] = qa_data.at[i-1, "question_num"]

    #Initialize lists and populate them with "speaker", "job_title", "utterance", and "question_number_inline" values from rows between last_a and i in qa_data.
    else:
      last_a = find_row(qa_data,"job_title", i)+ 1
      for j in range(last_a, i):
        name_list.append(qa_data["speaker"][j])
        bank_list.append(qa_data["job_title"][j])
        question_list.append(qa_data["utterance"][j])
       # quest_num_list.append(str(qa_data["question_number_inline"][j]))
 # Assign the lists name_list, bank_list, question_list, and quest_num_list to the "analyst", "analyst_bank", "metadata_question", and "question_num" columns at row i in qa_data, respectively.
    qa_data.at[i,"analyst"]=name_list
    qa_data.at[i,"analyst_bank"]=bank_list
    qa_data.at[i,"metadata_question"]=question_list
    qa_data.at[i,"question_num"]=quest_num_list

KeyError: 4

In [28]:
qa_data.head()

,speaker,job_title,utterance,call_date,financial_quarter,source_file,ex_dummy,category,question_tokenised_data,question_cleaned,count,analyst,analyst_bank,metadata_question,question_num
0,Chis Hallam,Goldman,"Chis Hallam, Goldman Sachs Yes. Good morning, ...",25 April 2023,1Q23,1q23-earnings-call-remarks.pdf,NaN,Goldman,"[chi, hallam, goldman, sachs, yes, good, morni...",[chis hallam goldman sachs yes good morning ev...,96,[],[],[],[]
1,Sergio P. Ermotti,UBS,"Okay. Thank you. On capital requirements, you ...",25 April 2023,1Q23,1q23-earnings-call-remarks.pdf,NaN,Goldman,"[okay, thank, you, capital, requirement, know,...",[okay thank you capital requirements know situ...,73,[],[],[],[]
2,Kian Abouhossein,JPMorgan,Yeah. Thanks. Just two questions. The first on...,25 April 2023,1Q23,1q23-earnings-call-remarks.pdf,NaN,JPMorgan,"[yeah, thanks, two, question, first, one, rela...",[yeah thanks two questions first one related b...,65,[],[],[],[]
3,Sarah Youngwood,UBS,"So, when we give you the 74%, we focused inten...",25 April 2023,1Q23,1q23-earnings-call-remarks.pdf,NaN,JPMorgan,"[give, focused, intentionally, viewed, economi...",[so give 74% focused intentionally viewed econ...,27,"[Sergio P. Ermotti, Kian Abouhossein]","[UBS, JPMorgan]","[Okay. Thank you. On capital requirements, you...",[]
4,Sergio P. Ermotti,UBS,"So on client retention, I – maybe let me reite...",25 April 2023,1Q23,1q23-earnings-call-remarks.pdf,NaN,JPMorgan,"[client, retention, maybe, let, reiterate, loo...",[client retention – maybe let reiterate look i...,80,[Sarah Youngwood],[UBS],"[So, when we give you the 74%, we focused inte...",[]


In [ ]:
for i in range(len(qa_data)):
  #if the first row is answer, then return all analyst columns as x, if the row is answer and there is not analyst, then popurate questioner analyst name and other data, otherwise pass the function
  if i ==0:
    qa_data.at[i,"analyst"] =qa_data.loc[i,"speaker"]
    qa_data.at[i,"analyst_title"] ="x"
    qa_data.at[i,"metadata_question"] ="x"
    qa_data.at[i,"question_num"]="x"
  elif qa_data["category"][i] is np.NaN and qa_data["analyst"][i]==[]:
    a = find_row_empty(qa_data,"category","analyst",i)
    qa_data.at[i,"analyst"] = qa_data.loc[a,"analyst"]
    qa_data.at[i,"analyst_title"] = qa_data.loc[a,"analyst_title"]
    qa_data.at[i,"metadata_question"] = qa_data.loc[a,"metadata_question"]
    qa_data.at[i,"question_num"] =qa_data.loc[a,"question_num"]
  else:
    continue


In [ ]:
#rename column
qa_data.rename(columns={"utterance":"metadata_answer","speaker":"Exective", "analyst_title":"Bank"},inplace=True)
qa_data.drop(columns=["category"],inplace=True)

In [ ]:
#reorganise column
qa_data=qa_data[["source_file","financial_quarter","call_date","Exective","question_num","metadata_question","question_cleaned","metadata_answer", "answer_cleaned","analyst","Bank"]]

KeyError: "['question_cleaned', 'metadata_answer', 'answer_cleaned'] not in index"

In [ ]:
print(qa_data["metadata_question"][2])

In [ ]:
qa_data = qa_data[(qa_data["metadata_question"]!="x") | (qa_data["metadata_answer"]==[])]
#

In [ ]:
qa_data.head(20)

# **Export the output as a csv file**

UBS QA section

In [ ]:
#export preprocessed data
preprocessed_qa_csv_path3 = "/content/ubs_qa_df_preprocessed.csv"
qa_data.to_csv(preprocessed_qa_csv_path3, index=False)

UBS management discussion

In [ ]:
#export preprocessed data
preprocessed_qa_csv_path4 = "/content/sample_data/ubs_management_df_preprocessed.csv"
ubs_manag_df.to_csv(preprocessed_qa_csv_path4, index=False)